In [1]:
import json
import theano
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras import models as md
from sklearn.metrics import auc
import import_ipynb
from my_libs.calc_features import clean_data
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
# import pandas as pd
# import numpy as np
# import warnings
# import re
# from datetime import datetime
# from collections import OrderedDict
# warnings.filterwarnings('default')

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb
importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\critical_temp_after_spryer.ipynb
importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\data_paths.ipynb


In [2]:
# возвращает модель, список признаков, по которому строилась модель и скейлер
def load_model(dir_name, target):
    model = md.load_model(dir_name+'/kerasNN_model'+'.h5')
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
    scaler = StandardScaler()
    scale_data = json.load(open(dir_name+'/scaler', "r"))
    scaler.mean_ = scale_data[0]
    scaler.scale_ = scale_data[1]
    return model, ls_need_col, scaler

In [3]:
def shift_columns_list(ls):
    ls = [ls[-3]]+[ls[-4]]+[ls[-2]]+[ls[-1]]+ls[:-4]
    return ls

In [4]:
def make_result_valid_file(input_file_name, dir_name, target, output_file_name):
    """Сохраняет файл с результатами """
    model, ls_need_col, scaler = load_model(dir_name, target)
    valid = clean_data(input_file_name, ls_need_col+[
        '№ партии', 
        'марка стали'])
    print(valid.shape)
    y_valid = valid[target]
    X_valid = valid[ls_need_col+['№ партии', 'марка стали']]
    X_valid_scale = scaler.transform(X_valid[ls_need_col])
    y_pred = model.predict(X_valid_scale)
    X_valid['predicted'] = y_pred
    X_valid[target] = y_valid
    X_valid['MAE'] = np.abs(X_valid[target]- X_valid['predicted'])
    X_valid['MSE'] = X_valid['MAE'].apply(lambda x: x*x)
    ls = shift_columns_list(list(X_valid.columns))
    X_valid = X_valid[ls]
    
    X_valid.to_excel(output_file_name)
    return X_valid

In [5]:
# X_valid = valid[ls_need_col+['№ плавки', '№ партии', 'марка стали']]
# X_valid.describe()

In [7]:
file_name = 'DATA/prepared/prepared_hard_valid.xlsx'
# target = 'Врем. сопротивление'
target = 'Предел текучести'
dir_name = 'DATA/MODELS_kerasNN_back/LIM YS BEST MF'
# dir_name = 'DATA/MODELS_kerasNN_back/LIM H BEST'
output_filename = 'DATA/modes_after_model/111__111.xlsx'

In [ ]:
df = make_result_valid_file(file_name, dir_name, target, output_filename)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [34]:
file_name = 'DATA/modes_before_model/127x12,5 37g2f _ input.xlsx'
# target = 'Врем. сопротивление'
target = 'Предел текучести'
dir_name = 'DATA/MODELS_kerasNN_back/LIM YS BEST MF'
# dir_name = 'DATA/MODELS_kerasNN_back/LIM H BEST'
output_filename = 'DATA/modes_after_model/NN 127x12,5 37g2f YS.xlsx'

In [35]:
df = make_result_valid_file(file_name, dir_name, target, output_filename)

(5, 35)
(5, 35)
(5, 35)


D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


KeyError: 'Предел текучести'

In [ ]:
valid = df

In [36]:
model, ls_need_col, scaler = load_model(dir_name, target)
y_valid = valid[target]
X_valid = valid[ls_need_col]
X_valid_scale = scaler.transform(X_valid)
y_pred = model.predict(X_valid_scale)
X_valid['predicted'] = y_pred
X_valid['real'] = y_valid
X_valid['MAE'] = np.abs(X_valid['real']- X_valid['predicted'])
X_valid['MSE'] = X_valid['MAE'].apply(lambda x: x*x)
ls = shift_columns_list(list(X_valid.columns))
X_valid = X_valid[ls]

NameError: name 'valid' is not defined

In [37]:
y_pred = y_pred[:,0]

NameError: name 'y_pred' is not defined

In [26]:
from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error

print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

corr 0.8699480998410973
r2 0.7265896144389831
explained_variance_score 0.7564886610658432
mean_squared_error 31.953936603797946
mean_absolute_error 4.6985904826703555
median_absolute_error 4.420819029516096
